Inspiration for this code is taken from Question and Answer model build by hugging face community and NER model creation using spacy.

In [ ]:
# Library for Google Colab
from google.colab import drive
# Mount Google Drive on kernel
drive.mount('/content/gdrive')
path = "/content/gdrive/My Drive/Colab Notebooks/Content"

Mounted at /content/gdrive


In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 34.4 MB/s 
     |████████████████████████████████| 163 kB 75.3 MB/s 
     |████████████████████████████████| 6.6 MB 55.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 34.5 MB/s 
     |████████████████████████████████| 212 kB 74.3 MB/s 
     |████████████████████████████████| 115 kB 73.7 MB/s 
     |████████████████████████████████| 127 kB 73.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 7.4 MB/s 


In [ ]:
!pip install -r Requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 125 kB 48.0 MB/s 
     |████████████████████████████████| 510 kB 72.1 MB/s 
     |████████████████████████████████| 1.3 MB 64.9 MB/s 
     |████████████████████████████████| 578.0 MB 16 kB/s 
ERROR: Could not find a version that satisfies the requirement numpy==1.23.1 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0rc1, 1.13.0rc2, 1.13.0, 1.13.1, 1.13.3, 1.14.0rc1, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0rc1, 1.15.0rc2, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0rc1, 1.16.0rc2, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0

In [ ]:
!pip install flask_ngrok
!pip install flask_cors
!pip install flask_restful

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 1.4 MB/s 


In [ ]:
from transformers import TFAutoModelForQuestionAnswering
from spacy.training.example import Example
from transformers import DefaultDataCollator
from transformers import create_optimizer
from flask_ngrok import run_with_ngrok
from transformers import AutoTokenizer
from transformers import pipeline
from datasets import load_dataset
from flask import Flask, request
from flask_restful import Api, Resource
from flask_cors import CORS, cross_origin
from tensorflow import keras
from tqdm.auto import tqdm
from os.path import exists
from os.path import exists
from spacy import displacy
from flask import Response
from joblib import dump
from joblib import load
import tensorflow as tf

import collections
import numpy as np
import evaluate
import random
import spacy
import glob
import json
import os

max_length = 384
stride = 128
n_best = 20
max_answer_length = 30

In [ ]:
def training(dataset, iterations):
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    ner = ''
    if 'ner' not in nlp.pipe_names:
        ner = nlp.add_pipe('ner', last=True)

    # add labels
    for annotations in dataset["annotations"]:
        for ent in annotations[-1]['entities']:
            ner.add_label(ent[-1])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(dataset["annotations"])
            losses = {}
            for annotations in dataset["annotations"]:
                doc = nlp.make_doc(annotations[0])
                example = Example.from_dict(doc, annotations[-1])
                nlp.update(
                    [example],
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

In [ ]:
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = eval(answers[sample_idx])
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": eval(ex["answer"])} for ex in examples]
    print("{},\n {}".format(predicted_answers, theoretical_answers))
    predicted_text = []
    references = []
    for element in range(len(predicted_answers)):
        predicted_text.append(predicted_answers[element]['prediction_text'])
        references.append(theoretical_answers[element]['answers']['text'])
    # return metric.compute(predictions=predicted_answers, references=theoretical_answers)
    return predicted_text, references

In [ ]:
# from flask_restful import Api, Resource

app = Flask(__name__)
# CORS(app, support_credentials=True)
# api = Api(app)
run_with_ngrok(app)


@app.route('/predict',methods=['POST'])
@cross_origin()
def predict():
  question_highlights = []
  context_highlights = []
  input_data = request.get_json()
  question_answerer = pipeline("question-answering", model=model_checkpoint)
  question = input_data['question']
  context = input_data['context']
  doc1 = legal_model(question)
  doc2 = legal_model(context)
  for ent in doc1.ents:
    question_highlights.append(ent.text)
  for ent in doc2.ents:
    context_highlights.append(ent.text)
    # print(ent.text, ent.start_char, ent.end_char, ent.label_)
  answer = question_answerer(question=question, context=context)
  output = {'question_highlights': question_highlights, 'context_highlights' : context_highlights, 'answer': answer, "question": question, "context": context}
  return output

References:
===========

Huggingface.co. 2022. Question answering - Hugging Face Course. [online] Available at: <https://huggingface.co/course/chapter7/7?fw=tf> [Accessed 17 September 2022].

Spacy.io. 2022. Trained Models & Pipelines. [online] Available at: <https://spacy.io/models#conventions> [Accessed 21 September 2022].

In [ ]:




if __name__ == '__main__':

    datafiles = {'train' : "train.csv", 'test' : "test.csv"}
    raw_datasets = load_dataset('csv', data_files=datafiles)
    print(raw_datasets['train'][0:2]['id'])
    model_checkpoint = "bert-base-cased"
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    train_dataset = raw_datasets["train"].map(
        preprocess_training_examples,
        batched=True,
        remove_columns=raw_datasets["train"].column_names,
    )
    print(len(raw_datasets["train"]), len(train_dataset))
    validation_dataset = raw_datasets["test"].map(
        preprocess_validation_examples,
        batched=True,
        remove_columns=raw_datasets["test"].column_names,
    )
    print(len(raw_datasets["test"]), len(validation_dataset))

    small_eval_set = raw_datasets["test"]
    trained_checkpoint = "distilbert-base-cased-distilled-squad"

    tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
    eval_set = small_eval_set.map(
        preprocess_validation_examples,
        batched=True,
        remove_columns=raw_datasets["test"].column_names,
    )
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
    eval_set_for_model.set_format("numpy")

    batch = {k: eval_set_for_model[k] for k in eval_set_for_model.column_names}
    trained_model = TFAutoModelForQuestionAnswering.from_pretrained(trained_checkpoint)

    outputs = trained_model(**batch)
    start_logits = outputs.start_logits.numpy()
    end_logits = outputs.end_logits.numpy()
    # metric = evaluate.load("squad")
    bleu = evaluate.load("bleu")
    predicted, theoretical = compute_metrics(start_logits, end_logits, eval_set, small_eval_set)
    results = bleu.compute(predictions=predicted, references=theoretical)
    print(results)
    model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
    data_collator = DefaultDataCollator(return_tensors="tf")
    tf_train_dataset = train_dataset.to_tf_dataset(
        columns=[
            "input_ids",
            "start_positions",
            "end_positions",
            "attention_mask",
            "token_type_ids",
        ],
        collate_fn=data_collator,
        shuffle=True,
        batch_size=16,
    )
    tf_eval_dataset = validation_dataset.to_tf_dataset(
        columns=["input_ids", "attention_mask", "token_type_ids"],
        collate_fn=data_collator,
        shuffle=False,
        batch_size=16,
    )
    if not exists("model.sav"):
        num_train_epochs = 5
        num_train_steps = len(tf_train_dataset) * num_train_epochs
        optimizer, schedule = create_optimizer(
            init_lr=2e-5,
            num_warmup_steps=0,
            num_train_steps=num_train_steps,
            weight_decay_rate=0.01,
        )
        model.compile(optimizer=optimizer)

        # Train in mixed-precision float16
        tf.keras.mixed_precision.set_global_policy("mixed_float16")
        model.fit(tf_train_dataset, epochs=num_train_epochs)
        model.save_pretrained("model.sav")
    else:
        model = TFAutoModelForQuestionAnswering.from_pretrained("model.sav")
    predictions = model.predict(tf_eval_dataset)
    predicted, theoretical = compute_metrics(
        predictions["start_logits"],
        predictions["end_logits"],
        validation_dataset,
        raw_datasets["test"],
    )
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=predicted, references=theoretical)
    model_file = "NER_model.joblib"
    if not exists(model_file):
        input_dataset = []
        classes_refined = []
        annotations_refined = []
        for name in glob.glob('annotated_dataset/*.json'):
            with open(name, 'r') as dataset:
                json_dataset = dataset.read()
                json_file = json.loads(json_dataset)
                input_dataset.append(json_file)
        for each_json in input_dataset:
            classes_refined = classes_refined + each_json["classes"]
            annotations_refined.extend(each_json["annotations"])
        json_loader = dict(zip(["classes", "annotations"], [classes_refined, annotations_refined]))
        legal_model = training(json_loader, 20)
        dump(legal_model, model_file)
    else:
        legal_model = load(model_file)
    app.run()

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ba28519eb4e9e086/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

...

{'ner': 3439.274428237491}
Starting iteration 1
{'ner': 1336.3256109406366}
Starting iteration 2
{'ner': 1189.1101407087385}
Starting iteration 3
{'ner': 1525.7905140764499}
Starting iteration 4
{'ner': 846.9846158416874}
Starting iteration 5
{'ner': 634.1663677690937}
Starting iteration 6
{'ner': 563.645440780617}
Starting iteration 7
{'ner': 932.1579937576962}
Starting iteration 8
{'ner': 551.982626273809}
Starting iteration 9
{'ner': 524.314436324226}
Starting iteration 10
{'ner': 564.3295619009688}
Starting iteration 11
{'ner': 502.0869322159818}
Starting iteration 12
{'ner': 476.52098042578643}
Starting iteration 13
{'ner': 377.33570126138403}
Starting iteration 14
{'ner': 339.4583168588715}
Starting iteration 15
{'ner': 343.05247491490144}
Starting iteration 16
{'ner': 336.9290511353118}
Starting iteration 17
{'ner': 303.0772129416119}
Starting iteration 18
{'ner': 369.93386832906447}
Starting iteration 19
{'ner': 290.3624718632853}
 * Serving Flask app "__main__" (lazy loading)


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://78a6-34-126-119-126.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a